In [2]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import geemap
import os

ee.Initialize()

def p2r(point, area_size = 100, field_size = 20):
    dist_lat = abs(area_size / 110.540)/1000
    dist_lon = abs(area_size / (111.320 * np.cos(point[0])))/1000
    area = {"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {},"geometry": {"type": "Polygon", "coordinates": [[[point[0] - dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] + dist_lat/2], [point[0] - dist_lon/2, point[1] + dist_lat/2], [point[0] - dist_lon/2, point[1] - dist_lat/2]]]}}]}
    dist_lat = abs(field_size / 110.540)/1000
    dist_lon = abs(field_size / (111.320 * np.cos(point[0])))/1000
    soccer={"type": "FeatureCollection", "features": [{"type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[point[0] - dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] - dist_lat/2], [point[0] + dist_lon/2, point[1] + dist_lat/2], [point[0] - dist_lon/2,point[1] + dist_lat/2], [point[0] - dist_lon/2, point[1] - dist_lat/2]]]}}]}
    coords_area = area['features'][0]['geometry']['coordinates']
    aoi_area = ee.Geometry.Polygon(coords_area)
    coords_soccer = soccer['features'][0]['geometry']['coordinates']
    aoi_soccer = ee.Geometry.Polygon(coords_soccer)
    return(aoi_area, aoi_soccer)

aoi_area, aoi_soccer = p2r([-1.0188, 6.6090])

In [3]:
start = time.time()

# https://gis.stackexchange.com/questions/372828/get-feature-values-from-imagecollection-to-list
date_start = '2020-01-01'
date_stop = '2020-12-31'
area = aoi_soccer.buffer(-10)

dataset = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(ee.Date(date_start),ee.Date(date_stop))

dataset = ee.ImageCollection("NASA/GPM_L3/IMERG_MONTHLY_V06").filterDate(ee.Date(date_start),ee.Date(date_stop)).select('precipitation')

# dataset = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
#                     .filterBounds(aoi_area)
#                     .filterDate(ee.Date(date_start),ee.Date(date_stop))
#                     .filter(ee.Filter.eq('instrumentMode', 'IW'))
#                     .sort('system:time_start'))

# dataset = (ee.ImageCollection("COPERNICUS/S2")
#                 .filterBounds(aoi_soccer)
#                 .filterDate(ee.Date(date_start),ee.Date(date_stop))
#                 .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
#                 .filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 20))
#                 .sort('system:time_start'))

acq_times = dataset.aggregate_array('system:time_start').getInfo()
avalaible_dates = [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
im_list = dataset.toList(dataset.size())

# Create list to hold precipitation values
precipitation_list = ee.List([])

 # Image reduction applied to each image.
def reduce_dataset_region(image, list):
    # Calculate mean of precipitation on defined area.
    local_precipitation_image = image.reduceRegion(
        reducer=ee.Reducer.median(),
        geometry=area,
        scale=10)
    return ee.List(list).add(local_precipitation_image)

# Apply region reduction to ImageCollection and return result in List
reduced_dataset = dataset.iterate(reduce_dataset_region, precipitation_list)

# Request server-side List  as dict
reduced_dataset_dict = reduced_dataset.getInfo()

rainfall = pd.DataFrame(reduced_dataset_dict)
rainfall.index = avalaible_dates

end = time.time()
print(np.round((end - start)/60, 3))

rainfall.dropna()

0.046


,precipitation
01/01/20,0.002908
02/01/20,0.005474
03/01/20,0.169181
04/01/20,0.120225
05/01/20,0.287003
06/01/20,0.339933
07/01/20,0.102657
08/01/20,0.028318
09/01/20,0.212263
10/01/20,0.216476


In [ ]:
# image = ee.ImageCollection("ECMWF/ERA5/MONTHLY").filterDate(ee.Date(date_start),ee.Date(date_stop)).first()
# out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
# filename = os.path.join(out_dir, 'DailyP.tif')
# geemap.ee_export_image(image, filename=filename, scale=90, region=aoi_area, file_per_band=True)

In [16]:
collection = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterDate(ee.Date(date_start),ee.Date(date_stop))
# collection = ee.ImageCollection("ECMWF/ERA5/MONTHLY").filterDate(ee.Date(date_start),ee.Date(date_stop)).select('mean_2m_air_temperature')
out_dir_col = os.path.join(os.path.expanduser('~'), 'Downloads')
geemap.ee_export_image_collection(collection, region=aoi_area, scale=90, out_dir=out_dir_col)